### Importação das bibliotecas que utilizaremos para trabalhar os dados e o arquivo chest_xray que contém as imagens para treinamento e teste

In [ ]:
# Importação das bibliotecas e imprimindo os arquivos que contém a pasta chest_xray
import pandas as pd 
import cv2            
import numpy as np         
import os                  
from random import shuffle
from tqdm import tqdm  
import scipy
import skimage
from skimage.transform import resize
#from keras.utils.np_utils import to_categorical
import tensorflow as tf
print(os.listdir("/home/vinicius/Documentos/UTFPR/IA/projeto_ia/Inceptionv3_Pneumonia/chest_xray/chest_xray"))

### Imprimindo os arquivos que contém chest_xray/train, esse arquivo possui as imagens de treinamento

In [ ]:
print(os.listdir("/home/vinicius/Documentos/UTFPR/IA/projeto_ia/Inceptionv3_Pneumonia/chest_xray/chest_xray/train"))


### Selecionando as imagens dos diretórios de treinamento e teste, e separando as na proporção de 80% treinamento e 20% teste.

In [ ]:
from sklearn.model_selection import train_test_split

TRAIN_DIR = "/home/vinicius/Documentos/UTFPR/IA/projeto_ia/Inceptionv3_Pneumonia/chest_xray/chest_xray/train"
TEST_DIR = "/home/vinicius/Documentos/UTFPR/IA/projeto_ia/Inceptionv3_Pneumonia/chest_xray/chest_xray/test"

filenames = tf.io.gfile.glob(str(TRAIN_DIR))
filenames.extend(tf.io.gfile.glob(str(TEST_DIR)))

train_filenames, test_filenames = train_test_split(filenames, test_size=0.2)
print(type(test_filenames))

### Função que buscar o rótulo(Normal ou PNEUMONIA) de cada imagem

In [ ]:
def get_data(Dir):
    X = []
    y = []
    for nextDir in os.listdir(Dir):
        if not nextDir.startswith('.'):
            if nextDir in ['NORMAL']:
                label = 0
            elif nextDir in ['PNEUMONIA']:
                label = 1
            else:
                label = 2
                
            temp = Dir + '/' + nextDir
            #tqdm mostra o progresso
            for file in tqdm(os.listdir(temp)):
                img = cv2.imread(temp + '/' + file)
                if img is not None:
                    img = skimage.transform.resize(img, (150, 150, 3))
                    #img_file = scipy.misc.imresize(arr=img_file, size=(299, 299, 3))
                    img = np.asarray(img)
                    X.append(img)
                    #print(X)
                    y.append(label)
                    #print(y)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [ ]:
#retorna as imagens de train em array
train_filenames = "".join(map(str,train_filenames ))
X_train, y_train = get_data(train_filenames)


In [ ]:
#retorna as imagens de teste em array
test_filenames = "".join(map(str,test_filenames ))
X_test , y_test = get_data(test_filenames)

### Imprime a dimensão do DataFrame

In [ ]:
print(X_train.shape,'\n',X_test.shape)
#X_train

In [ ]:
print(y_train.shape,'\n',y_test.shape)

### Converte uma classe vetor em matrix de classe binaria

In [ ]:
from keras.utils.np_utils import to_categorical
#to_categorical, transforma a entrada em binario, 2 eh o numero de classe
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [ ]:
#Cria uma lista com o nome das imagens
#listdir tras o nome de todos objetos contidos no diretorio
Pimages = os.listdir(TRAIN_DIR + '/' + "PNEUMONIA")
#print(Pimages)
Nimages = os.listdir(TRAIN_DIR + '/' + "NORMAL")

### Plotando as imagens de Raio X de Não Pneumonia e Pneumonia dos pacientes

In [ ]:
import matplotlib.pyplot as plt
def plotter(i):
    imagep1 = cv2.imread(TRAIN_DIR + '/PNEUMONIA/' + Pimages[i])
    #print(imagep1)
    imagep1 = skimage.transform.resize(imagep1, (150,150,3), mode = 'reflect')
    imagen1 = cv2.imread(TRAIN_DIR + '/NORMAL/' + Nimages[i])
    imagen1 = skimage.transform.resize(imagen1, (150,150,3), mode = 'reflect')
    pair = np.concatenate((imagen1, imagep1), axis = 1)
    print("(Left) - No Pneumonia Vs (Right) - Pneumonia")
    print("-----------------------------------------------------------------------------------------------------------------------------------")
    plt.figure(figsize=(10,5))
    plt.imshow(pair)
    plt.show()
for i in range(12,15):
    plotter(i)

### Reduz a taxa de aprendizagem quando uma métrica para de melhorar, input da taxa de aprendizagem

In [ ]:
from keras.callbacks import ReduceLROnPlateau , ModelCheckpoint , LearningRateScheduler
lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.001, epsilon=0.0001, patience=1, verbose=1)

### Chamada para salvar o modelo Keras ou pesos do modelo com alguma frequência no arquivo transferlearning_weights.hdf5

In [ ]:
filepath="transferlearning_weights.hdf5"
#save the model after every epoch
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

### Importando biblioteca keras

In [ ]:
from keras.models import Sequential , Model
from keras.layers import Dense , Activation
from keras.layers import Dropout , GlobalAveragePooling2D
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , RMSprop , Adadelta , Adam
from keras.layers import Conv2D , BatchNormalization
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
#K.set_image_dim_ordering('th')
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

### Padronizando a dimensão do Dataframe

In [ ]:
X_train=X_train.reshape(344,150,150,3)
X_test=X_test.reshape(1390,150,150,3)

In [ ]:
print(np.shape(X_train))

### Importando InceptionV3 de Keras, com os pesos do imagenet (pré-treinamento)


In [ ]:
from keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(weights='imagenet', include_top=False )


In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras import backend as K


x = base_model.output
x = Dropout(0.5)(x)

x = GlobalMaxPooling2D()(x)

x = Dense(128, activation='relu')(x)

predictions = Dense(2, activation='sigmoid')(x)

In [ ]:
from keras.models import Model
model= Model(base_model.input,predictions)

In [ ]:
model.compile(loss='binary_crossentropy',
                  optimizer='adam', 
                  metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
batch_size = 32
epochs = 500
Learning_rate = 0.001
Dropout = 0.5


In [ ]:
history = model.fit(x= X_train, y = y_train, validation_data = (X_test , y_test) ,callbacks=[lr_reduce,checkpoint] ,epochs = epochs,batch_size = batch_size)
# fit(self, x, y, batch_size, nb_epoch, verbose, callbacks, validation_split, validation_data, shuffle, class_weight, sample_weight, initial_epoch
#history = model.fit(X_train, y_train,  (X_test , y_test) ,[lr_reduce,checkpoint] , epochs)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
pred = model.predict(X_test)
pred = np.argmax(pred,axis = 1) 
y_true = np.argmax(y_test,axis = 1)

In [ ]:
print(y_true)

In [ ]:
CM = confusion_matrix(y_true, pred)
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(5, 5))
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true, pred))
print(classification_report(y_true, pred))